In [ ]:
from google.colab import drive
drive.mount('/gdrive')


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/projet_1_ANN


In [ ]:
import numpy as np
# Fix randomness and hide warnings
seed = 42
from tensorflow.keras.applications.resnet50 import preprocess_input

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
np.random.seed(seed)

import logging

import random
random.seed(seed)

# Import tensorflow
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print(tf.__version__)

# Import other libraries
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import seaborn as sns

2.14.0




## data processing_laure

In [ ]:
loaded_data = np.load('public_data.npz', allow_pickle=True)

# Inspect the target
X = loaded_data['data']
y = loaded_data['labels']
print('Counting occurrences of target classes:')
print(pd.DataFrame(y, columns=['label'])['label'].value_counts())

Counting occurrences of target classes:
healthy      3199
unhealthy    2001
Name: label, dtype: int64


In [ ]:
y_data = pd.DataFrame(y, columns=['label'])
y_data['label'] = np.where(y_data['label'] == 'healthy',0,1)
y  = np.array(y_data['label'])
print(pd.DataFrame(y, columns=['label'])['label'].value_counts())


0    3199
1    2001
Name: label, dtype: int64


In [ ]:
data = X

In [ ]:
outlier_shrek = []
nb_outlier_shrek = [] #positions of the shreks


for i in range (len(data)):
  if np.mean(data[i])==np.mean(data[137]):
    outlier_shrek.append(data[i])
    nb_outlier_shrek.append(i)

np.shape(outlier_shrek)

(98, 96, 96, 3)

In [ ]:
outlier_trololo = []
nb_outlier_trololo = [] #positions of the shreks


for i in range (len(data)):
  if np.mean(data[i])==np.mean(data[1988]):
    outlier_trololo.append(data[i])
    nb_outlier_trololo.append(i)

np.shape(outlier_trololo)

(98, 96, 96, 3)

In [ ]:
new_data = []
new_labels_bool = []
for i in range(len(data)):
  if np.isin(i,nb_outlier_trololo) == False and np.isin(i,nb_outlier_shrek) == False:
    new_data.append(data[i])
    new_labels_bool.append(y[i])

print("the shape of data without outlier is : ", np.shape(new_data))

the shape of data without outlier is :  (5004, 96, 96, 3)


In [ ]:
X = np.array(new_data)
y = np.array(new_labels_bool)

In [ ]:
images = []
for i in range(len(new_data)) :
    img = new_data[i]
# Normalize image pixel values to a float range [0, 1]
    img = (img / 255).astype(np.float32)
    if img is not None:
            images.append(img)

In [ ]:
import cv2
import numpy as np

def supprimer_doublons_avec_labels(images, labels):
    # Créer une liste pour stocker les indices des images uniques
    indices_uniques = []

    # Créer un dictionnaire pour suivre les images déjà vues
    images_vues = {}

    for i, image in enumerate(images):
        # Convertir l'image en une chaîne d'octets pour pouvoir l'utiliser comme clé dans le dictionnaire
        image_str = image.tobytes()

        # Vérifier si l'image a déjà été rencontrée
        if image_str not in images_vues:
            # Ajouter l'index de l'image unique à la liste
            indices_uniques.append(i)

            # Enregistrer l'image dans le dictionnaire avec sa représentation en chaîne d'octets comme clé
            images_vues[image_str] = True

    # Retourner les images uniques et les labels correspondants
    images_uniques = [images[i] for i in indices_uniques]
    labels_uniques = [labels[i] for i in indices_uniques]

    return images_uniques, labels_uniques

# Exemple d'utilisation
# Supposons que images soit une liste d'images et labels soit une liste de labels correspondants
# Assure-toi que les images sont des tableaux NumPy


# Maintenant, images contient des images uniques et labels contient les labels correspondants


In [ ]:
X = np.array(images)
y = np.array(new_labels_bool)

In [ ]:
X_new, y_new = supprimer_doublons_avec_labels(X, y)


In [ ]:
X = np.array(X_new)
y = np.array(y_new)

In [ ]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, random_state=seed, test_size=0.05, stratify=y)

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, random_state=seed, test_size=0.1, stratify=y_train_val)


In [ ]:
np.savez('XYtest', X_test=X_test, y_test=y_test)
np.savez('XYval', X_val=X_val, y_val=y_val)
np.savez('XYtrainprocess', X_train=X_train, y_train=y_train)

In [ ]:
XYtest = np.load('XYtest.npz', allow_pickle=True)
XYval= np.load('XYval.npz', allow_pickle=True)

X_test = XYtest['X_test']
y_test = XYtest['y_test']
#y_test = 1 - y_test

X_val = XYval['X_val']
y_val = XYval['y_val']
#y_val = 1 - y_val

XYtrain = np.load('XYtrainprocess.npz', allow_pickle=True)
X_train = XYtrain['X_train']
y_train = XYtrain['y_train']
#y_train = 1 - y_train




In [ ]:
translation = tf.keras.Sequential([
  tfkl.RandomTranslation(0.2,0.2),
])
X_train_translation = translation(X_train)



rotation = tf.keras.Sequential([
  tfkl.RandomRotation(0.2),
])
X_train_rotation = rotation(X_train)



zoom = tf.keras.Sequential([
  tfkl.RandomZoom(0.2),
])
X_train_zoom = zoom(X_train)




X_train = np.concatenate((X_train,X_train_rotation,X_train_zoom),axis= 0)
y_train = np.concatenate((y_train,y_train,y_train),axis = 0)


print(X_train.shape)
print(y_train.shape)

(12438, 96, 96, 3)
(12438,)


## Model

In [ ]:
# Define key model parameters
input_shape = X_train.shape[1:]  # Input shape for the model
output_shape = 1 # Output shape for the model

# cf FNN_all pour plus d'info
batch_size = 128                # Batch size for training
epochs = 200
dropout_rate = 1/4

# Print the defined parameters
print("Epochs:", epochs)
print("Batch Size:", batch_size)
print("Input Shape:", input_shape)
print("Output Shape:", output_shape)

Epochs: 200
Batch Size: 128
Input Shape: (96, 96, 3)
Output Shape: 1


In [ ]:


resnet = tf.keras.applications.resnet50.ResNet50(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=(96,96,3),
    pooling='avg',
    classes=1000,

)


resnet_v2_50 = tf.keras.applications.ResNet50V2(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=(96,96,3),
    pooling='avg',
    classes=1000
)


xception_model = tf.keras.applications.Xception(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=(96, 96, 3),
    pooling='avg',
    classes=1000
)

convnext = tf.keras.applications.convnext.ConvNeXtLarge(
    model_name='convnext_large',
    include_top=False,
    include_preprocessing=True,
    weights='imagenet',
    input_tensor=None,
    input_shape=(96,96,3),
    pooling='avg',
    classes=1000,
    classifier_activation='softmax'
)


lr_patience = 7 # on improve pas au bout de 5 epochs on doit etre trop grossier dans le learning rate pour aller chercher les minimums
lr_scheduler = tfk.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy',     # Metric to monitor (validation mean squared error in this case)
    patience=lr_patience,  # Number of epochs with no improvement after which learning rate will be reduced
    factor=0.5,          # Factor by which the learning rate will be reduced (0.999 in this case) new_lr = lr * factor
    mode='max',            # Mode to decide when to reduce learning rate ('min' means reduce when metric stops decreasing)
    min_lr=0            # Minimum learning rate
)
# on peut l'ajouter à la liste des callbacks avec le early stopping et on utilise lee call back dans le fit
callbacks = [lr_scheduler]



def build_model1(input_shape, output_shape, cnn,seed=seed):
    tf.random.set_seed(seed)
    #cnn.trainable =False
    # Build the neural network layer by layer
    # input layer juste mettre le nombre de neurone de la premiere couche
    for layer in cnn.layers:
        layer.trainable = True
        layer.kernel_regularizer=tf.keras.regularizers.l2()




    preprocessing = tf.keras.Sequential([
        tfkl.RandomFlip("horizontal"),
         tfkl.RandomRotation(0.2),
        tfkl.RandomTranslation(0.2,0.2),
        tfkl.RandomZoom(0.2),
        tfkl.RandomBrightness(0.2, value_range=(0,1)),
       # tfkl.RandomContrast(0.3)
    ], name='preprocessing')

   # Build the neural network layer by layer


    input_layer = tfkl.Input(shape=input_shape, name='Input')



    preprocessing1 = preprocessing(input_layer)


    print(preprocessing1.shape)


    x = cnn(preprocessing1)




    # FNN
    #### on commence le fnn
    classifier_layer=tfkl.Dense(
        units=256, # 120 neurones
        activation='relu',
        kernel_regularizer=tf.keras.regularizers.l2(),
        name='dense1'
    )(x)
    dropout = tfkl.Dropout(dropout_rate, seed=seed)(classifier_layer)



    # l'output layer
    output_layer = tfkl.Dense(
        units=output_shape, # le nombre de classe
        activation='sigmoid', # pour la classification
        kernel_regularizer=tf.keras.regularizers.l2(),
        name='Output'
    )(dropout)


    # Create a Model connecting input and output
    model = tfk.Model(inputs=input_layer, outputs=output_layer , name='model1')

# Compile the model with Categorical Cross-Entropy loss and Adam optimizer
    model.compile(loss=tfk.losses.BinaryFocalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])


    return model


  #classifier_layer = tfkl.Dense(
   #     units=128,
    #    activation='relu',
     #   kernel_regularizer=tf.keras.regularizers.l2(),
      #  name='dense2'
    #)(dropout)
    #dropout = tfkl.Dropout(dropout_rate, seed=seed)(classifier_layer)




## Train and save the model


In [ ]:
model = build_model1(input_shape, output_shape,convnext)
model.summary()

(None, 96, 96, 3)
Model: "model1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 96, 96, 3)]       0         
                                                                 
 preprocessing (Sequential)  (None, 96, 96, 3)         0         
                                                                 
 convnext_large (Functional  (None, 1536)              196230336 
 )                                                               
                                                                 
 dense1 (Dense)              (None, 256)               393472    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 Output (Dense)              (None, 1)                 257       
                                          

In [ ]:
#from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications.resnet_v2 import preprocess_input as preprocess_input_50


from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight('balanced', classes=np.unique(y_train),y=y_train)

history = model.fit(
    x =X_train, # We need to apply the preprocessing thought for the MobileNetV2 network
    y = y_train,
    class_weight=dict(enumerate(class_weights)),
    batch_size = 32,
    epochs = 200,
    validation_data = (X_val, y_val), # We need to apply the preprocessing thought for the MobileNetV2 network
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=30, restore_best_weights=True),lr_scheduler]
).history




Epoch 1/200
389/389 [==============================] - 173s 276ms/step - loss: 3.0398 - accuracy: 0.4952 - val_loss: 1.8825 - val_accuracy: 0.3688 - lr: 1.0000e-04
Epoch 2/200
389/389 [==============================] - 92s 236ms/step - loss: 1.3147 - accuracy: 0.4761 - val_loss: 0.8998 - val_accuracy: 0.6312 - lr: 1.0000e-04
Epoch 3/200
389/389 [==============================] - 91s 234ms/step - loss: 0.6801 - accuracy: 0.5023 - val_loss: 0.5073 - val_accuracy: 0.6312 - lr: 1.0000e-04
Epoch 4/200
389/389 [==============================] - 91s 234ms/step - loss: 0.4100 - accuracy: 0.4759 - val_loss: 0.3303 - val_accuracy: 0.6312 - lr: 1.0000e-04
Epoch 5/200
389/389 [==============================] - 91s 234ms/step - loss: 0.2870 - accuracy: 0.4847 - val_loss: 0.2516 - val_accuracy: 0.3688 - lr: 1.0000e-04
Epoch 6/200
389/389 [==============================] - 91s 234ms/step - loss: 0.2306 - accuracy: 0.4638 - val_loss: 0.2139 - val_accuracy: 0.3688 - lr: 1.0000e-04
Epoch 7/200
389/389 [

KeyboardInterrupt: ignored

In [ ]:
model.save('1_model_xception_V2')

## prediction

In [ ]:
model = tfk.models.load_model('mobilFT1.3ft43')
# Predict labels for the entire test set
predictions = model.predict(preprocess_input(X_test*255), verbose=0)

# Display the shape of the predictions
print("Predictions Shape:", y_test.shape)

Predictions Shape: (243,)


In [ ]:
threshold = 0.5

# Assuming predictions is your array of probabilities
predictions = model.predict(preprocess_input(X_test*255), verbose=0)

# Convert probabilities to 0 or 1 based on the threshold
binary_predictions = (predictions >= threshold).astype(int)

In [ ]:
# Compute the confusion matrix
cm = confusion_matrix(y_test, binary_predictions[:, 0])

# Compute classification metrics
accuracy = accuracy_score(y_test,binary_predictions[:, 0])
precision = precision_score( y_test,binary_predictions[:, 0], average='macro')
recall = recall_score(y_test,binary_predictions[:, 0], average='macro')
f1 = f1_score(y_test,binary_predictions[:, 0], average='macro')

# Display the computed metrics
print('Accuracy:', accuracy.round(4))
print('Precision:', precision.round(4))
print('Recall:', recall.round(4))
print('F1:', f1.round(4))
cm
# Plot the confusion matrix
#plt.figure(figsize=(10, 8))
#sns.heatmap(cm.T, xticklabels=list(labels.values()), yticklabels=list(labels.values()), cmap='Blues')
#plt.xlabel('True labels')
#plt.ylabel('Predicted labels')
#plt.show()

Accuracy: 0.6296
Precision: 0.3148
Recall: 0.5
F1: 0.3864


array([[153,   0],
       [ 90,   0]])

## finetuning

In [ ]:
# Re-load the model after transfer learning
ft_model = tfk.models.load_model('mobilFT1.3ft43')


# Assuming ft_model is a loaded ResNet model
ft_model = tfk.models.load_model('mobilFT1.3ft43')

# Print the names of all layers in the model
for layer in ft_model.layers:
    print(layer.name)



Input
preprocessing
resnet50v2
dense1
dropout_16
Output


In [ ]:
# Set all MobileNetV2 layers as trainable
ft_model.get_layer('resnet50v2').trainable = True
#for i, layer in enumerate(ft_model.get_layer('mobilenetv2_1.00_224').layers):
 #  print(i, layer.name, layer.trainable)


In [ ]:
lr_patience = 1 # on improve pas au bout de 5 epochs on doit etre trop grossier dans le learning rate pour aller chercher les minimums
lr_scheduler = tfk.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy',     # Metric to monitor (validation mean squared error in this case)
    patience=lr_patience,  # Number of epochs with no improvement after which learning rate will be reduced
    factor=0.5,          # Factor by which the learning rate will be reduced (0.999 in this case) new_lr = lr * factor
    mode='max',            # Mode to decide when to reduce learning rate ('min' means reduce when metric stops decreasing)
    min_lr=0            # Minimum learning rate
)

In [ ]:
# Freeze first N layers, e.g., until the 133rd one
N = 20
for i, layer in enumerate(ft_model.get_layer('resnet50v2').layers[:N]):
  layer.trainable=False
for i, layer in enumerate(ft_model.get_layer('resnet50v2').layers):
   print(i, layer.name, layer.trainable)
ft_model.summary()

0 input_49 False
1 conv1_pad False
2 conv1_conv False
3 pool1_pad False
4 pool1_pool False
5 conv2_block1_preact_bn False
6 conv2_block1_preact_relu False
7 conv2_block1_1_conv False
8 conv2_block1_1_bn False
9 conv2_block1_1_relu False
10 conv2_block1_2_pad False
11 conv2_block1_2_conv False
12 conv2_block1_2_bn False
13 conv2_block1_2_relu False
14 conv2_block1_0_conv False
15 conv2_block1_3_conv False
16 conv2_block1_out False
17 conv2_block2_preact_bn False
18 conv2_block2_preact_relu False
19 conv2_block2_1_conv False
20 conv2_block2_1_bn True
21 conv2_block2_1_relu True
22 conv2_block2_2_pad True
23 conv2_block2_2_conv True
24 conv2_block2_2_bn True
25 conv2_block2_2_relu True
26 conv2_block2_3_conv True
27 conv2_block2_out True
28 conv2_block3_preact_bn True
29 conv2_block3_preact_relu True
30 conv2_block3_1_conv True
31 conv2_block3_1_bn True
32 conv2_block3_1_relu True
33 conv2_block3_2_pad True
34 conv2_block3_2_conv True
35 conv2_block3_2_bn True
36 conv2_block3_2_relu True


In [ ]:
# Compile the model
# Compile the model with Categorical Cross-Entropy loss and Adam optimizer
ft_model.compile(loss=tfk.losses.BinaryFocalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate=0.00001), metrics=['accuracy'])

#ft_model.compile(loss=tfk.losses.BinaryFocalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate=0.001), metrics='binary_accuracy')

In [ ]:
print(X_train.shape)
print(y_train.shape)


In [ ]:

# Fine-tune the model
ft_history = ft_model.fit(
    x = preprocess_input(X_train*255), # We need to apply the preprocessing thought for the MobileNetV2 network
    y = y_train,
    batch_size = 64,
    epochs =200,
    validation_data = (preprocess_input_50(X_val*255), y_val), # We need to apply the preprocessing thought for the resnet network
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=5, restore_best_weights=True),lr_scheduler]
).history

Epoch 1/200
65/65 [==============================] - 37s 210ms/step - loss: 0.2171 - accuracy: 0.6266 - val_loss: 0.2762 - val_accuracy: 0.6312 - lr: 1.0000e-05
Epoch 2/200
65/65 [==============================] - 12s 177ms/step - loss: 0.2066 - accuracy: 0.6623 - val_loss: 0.2355 - val_accuracy: 0.6291 - lr: 1.0000e-05
Epoch 3/200
65/65 [==============================] - 12s 178ms/step - loss: 0.2017 - accuracy: 0.6838 - val_loss: 0.2379 - val_accuracy: 0.6312 - lr: 5.0000e-06
Epoch 4/200
65/65 [==============================] - 12s 177ms/step - loss: 0.2001 - accuracy: 0.6843 - val_loss: 0.2407 - val_accuracy: 0.6312 - lr: 2.5000e-06
Epoch 5/200
65/65 [==============================] - 12s 177ms/step - loss: 0.1971 - accuracy: 0.6896 - val_loss: 0.2416 - val_accuracy: 0.6312 - lr: 1.2500e-06
Epoch 6/200
65/65 [==============================] - 12s 179ms/step - loss: 0.1979 - accuracy: 0.6857 - val_loss: 0.2425 - val_accuracy: 0.6312 - lr: 6.2500e-07


In [ ]:
ft_model.save('mobilFT1.3ft43_ft')

In [ ]:
threshold = 0.5

# Assuming predictions is your array of probabilities
predictions = model.predict(preprocess_input(X_test*255), verbose=0)

# Convert probabilities to 0 or 1 based on the threshold
binary_predictions = (predictions >= threshold).astype(int)